In [1]:
import pandas as pd

data = pd.read_csv("groceriesSynthetic.tsv", sep = "\t")
df = pd.DataFrame(data)
df_original = pd.DataFrame(data)

In [2]:
#Checking if all the values in currency are EUR
print(df["currency"].nunique() == "EUR")

False


In [3]:
rows_with_other_currencies = df[df["currency"] != "EUR"]
print(rows_with_other_currencies)

         id formattedDate               merchantName  \
804     704    06-03-2023                        NaN   
1362   1384    25-06-2023                        NaN   
1806   1381    04-04-2023                        NaN   
2530     18    16-03-2023                        NaN   
3066   1110    01-12-2022                        NaN   
...     ...           ...                        ...   
96983   477    15-04-2023                        NaN   
97047  1763    16-08-2023                        NaN   
97815  1218    05-06-2023                        NaN   
97902  1446    30-08-2023                        NaN   
99425   985    15-07-2023  TALLINN anyone Debrashire   

                                          remittanceInfo   value currency  
804    292384******9276 either JARVE SELVERI ISETEENI...  -10.74      USD  
1362   (..3895) carry any WoltItaemerenkatu somebody new    -5.7      SEK  
1806   (..6464) always sign protect everybody improve...  -22.44      SEK  
2530   (..5274) scene p

In [4]:
pip install CurrencyConverter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from currency_converter import CurrencyConverter
c = CurrencyConverter()

def currency_to_EUR(row):
    print(row)
    if(row["currency"] != "EUR"):
        try:
            conversion = c.convert(row["value"], row["currency"], "EUR")
            return round(conversion,2)
        except:
            return None

for index, row in df.iterrows():
    row["value"] = currency_to_EUR(row)
    df.at[index, "value"] = row["value"]

id                                                             1269
formattedDate                                            24-07-2023
merchantName                                                    NaN
remittanceInfo    567176******6366 toward information MAKSIMARKE...
value                                                        -13.59
currency                                                        EUR
Name: 0, dtype: object
id                                                              319
formattedDate                                            14-06-2023
merchantName                                                    NaN
remittanceInfo    157466******5153 pay LIDL own TARTU sound behi...
value                                                         -0.89
currency                                                        EUR
Name: 1, dtype: object
id                                                              981
formattedDate                                            28-10-2022
me

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
rows_with_other_currencies = df[df["currency"] != "EUR"]
print(rows_with_other_currencies)

In [ ]:
df = df.drop(columns="currency")